# [6] Wardバッチ処理 - Google Colab版

A100 GPUを使用して高速にward検出を実行します。

## 事前準備（ローカルPC）
Google Driveに以下をアップロード:
1. `models/best.pt` -> `MyDrive/pyLoL/models/best.pt`
2. `C:\dataset_20260105` -> `MyDrive/pyLoL/dataset_20260105/` (フォルダごと)
3. `data/timeline/` -> `MyDrive/pyLoL/timeline/`

## 処理フロー
1. **YOLO推論**: GPUバッチ推論（BATCH_SIZE=64）
2. **クラスタリング**: 同一wardをグループ化
3. **タイムライン統合**: Riot APIのwardイベントとマッチング

In [ ]:
#cell-1: 環境セットアップ
# Driveマウント
from google.colab import drive
drive.mount('/content/drive')

# ultralytics インストール
!pip install -q ultralytics

# GPU確認
!nvidia-smi --query-gpu=name,memory.total --format=csv

In [ ]:
#cell-2: インポート + 設定
from pathlib import Path
import numpy as np
import pandas as pd
import csv
import json
from collections import defaultdict
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple
import time

from tqdm import tqdm
from ultralytics import YOLO
import torch
from scipy.optimize import linear_sum_assignment

# === Google Drive パス設定 ===
DRIVE_ROOT = Path("/content/drive/MyDrive/pyLoL")
MODEL_PATH = DRIVE_ROOT / "models" / "best.pt"
DATASET_DIR = DRIVE_ROOT / "dataset_20260105"
TIMELINE_DIR = DRIVE_ROOT / "timeline"

# === 推論設定 ===
CONFIDENCE_THRESHOLD = 0.6
IMAGE_SIZE = 512
BATCH_SIZE = 64  # A100用（40GB VRAM）

# === クラスタリング設定 ===
DISTANCE_THRESHOLD = 0.01
MIN_FRAMES = 3
GAP_TOLERANCE = 10

# === タイムライン統合設定 ===
FRAME_TOLERANCE = 10
DEFAULT_MS_PER_FRAME = 376
MIN_CONFIDENCE_DETECTION_ONLY = 0.5
RIVER_SIGHT_POSITIONS = [(362, 332), (151, 178)]
RIVER_SIGHT_RADIUS = 10
RIVER_SIGHT_DURATION_MIN_MS = 75000
RIVER_SIGHT_DURATION_MAX_MS = 93000

# パス確認
print(f"モデル: {MODEL_PATH}")
print(f"  存在: {MODEL_PATH.exists()}")
print(f"\nデータセット: {DATASET_DIR}")
print(f"  存在: {DATASET_DIR.exists()}")
if DATASET_DIR.exists():
    match_dirs = sorted(DATASET_DIR.glob("JP1-*"))
    print(f"  試合数: {len(match_dirs)}")
print(f"\nタイムライン: {TIMELINE_DIR}")
print(f"  存在: {TIMELINE_DIR.exists()}")

# GPU確認
if torch.cuda.is_available():
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"\nGPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {vram_gb:.1f} GB")
    print(f"バッチサイズ: {BATCH_SIZE}")
else:
    print("\n警告: GPUが利用できません")
    BATCH_SIZE = 1

# モデル読み込み
model = YOLO(str(MODEL_PATH))
print(f"\nモデル読み込み完了")
print(f"クラス: {model.names}")

In [ ]:
#cell-3: データ構造 + 関数定義

# === データ構造 ===
@dataclass
class Detection:
    frame: int
    class_id: int
    class_name: str
    x: float
    y: float
    w: float
    h: float
    confidence: float

@dataclass
class Ward:
    ward_id: int
    class_id: int
    class_name: str
    x: float
    y: float
    frame_start: int
    frame_end: int
    detections: List[Detection] = field(default_factory=list)
    matched: bool = False
    timeline_ward_id: Optional[int] = None

    @property
    def confidence_avg(self) -> float:
        return sum(d.confidence for d in self.detections) / len(self.detections) if self.detections else 0.0

    @property
    def detection_count(self) -> int:
        return len(self.detections)

    @property
    def team(self) -> str:
        return "red" if "enemy" in self.class_name else "blue"

    @property
    def is_stealth_ward(self) -> bool:
        return "stealth_ward" in self.class_name

    @property
    def is_control_ward(self) -> bool:
        return "control_ward" in self.class_name

    @property
    def x_pixel(self) -> int:
        return int(self.x * 512)

    @property
    def y_pixel(self) -> int:
        return int(self.y * 512)

@dataclass
class TimelineWardEvent:
    timeline_ward_id: int
    event_type: str
    timestamp: int
    ward_type: str
    participant_id: int
    team: str
    frame_expected: int

    @property
    def is_stealth_ward(self) -> bool:
        return self.ward_type in ("YELLOW_TRINKET", "SIGHT_WARD", "BLUE_TRINKET")

    @property
    def is_control_ward(self) -> bool:
        return self.ward_type == "CONTROL_WARD"

@dataclass
class MatchedWard:
    ward_id: int
    timeline_ward_id: Optional[int]
    class_name: str
    ward_type: str
    team: str
    x_pixel: int
    y_pixel: int
    x_normalized: float
    y_normalized: float
    frame_start: int
    frame_end: int
    confidence_avg: float
    creator_id: Optional[int]
    timestamp_placed: Optional[int]
    timestamp_killed: Optional[int]
    match_status: str

# === 推論関数 ===
def run_inference(model, match_dir: Path, conf: float = CONFIDENCE_THRESHOLD,
                  batch_size: int = BATCH_SIZE) -> List[Detection]:
    frame_dir = match_dir / "0"
    if not frame_dir.exists():
        return []

    frame_files = sorted(frame_dir.glob("*.png"), key=lambda p: int(p.stem))
    if not frame_files:
        return []

    detections = []
    for i in range(0, len(frame_files), batch_size):
        batch_files = frame_files[i:i+batch_size]
        batch_paths = [str(p) for p in batch_files]
        frame_nums = [int(p.stem) for p in batch_files]

        results = model(batch_paths, imgsz=IMAGE_SIZE, conf=conf, verbose=False)

        for frame_num, result in zip(frame_nums, results):
            for box in result.boxes:
                detections.append(Detection(
                    frame=frame_num,
                    class_id=int(box.cls[0]),
                    class_name=model.names[int(box.cls[0])],
                    x=box.xywhn[0][0].item(),
                    y=box.xywhn[0][1].item(),
                    w=box.xywhn[0][2].item(),
                    h=box.xywhn[0][3].item(),
                    confidence=float(box.conf[0])
                ))
    return detections

# === クラスタリング ===
def cluster_detections(detections: List[Detection]) -> List[Ward]:
    if not detections:
        return []

    sorted_detections = sorted(detections, key=lambda d: d.frame)
    active_wards: Dict[int, List[Ward]] = defaultdict(list)
    completed_wards = []
    next_ward_id = 1

    for det in sorted_detections:
        matched = False
        for ward in active_wards[det.class_id]:
            dist = np.sqrt((det.x - ward.detections[-1].x)**2 + (det.y - ward.detections[-1].y)**2)
            if dist < DISTANCE_THRESHOLD and det.frame - ward.frame_end <= GAP_TOLERANCE:
                ward.detections.append(det)
                ward.frame_end = det.frame
                ward.x = sum(d.x for d in ward.detections) / len(ward.detections)
                ward.y = sum(d.y for d in ward.detections) / len(ward.detections)
                matched = True
                break

        if not matched:
            new_ward = Ward(
                ward_id=next_ward_id, class_id=det.class_id, class_name=det.class_name,
                x=det.x, y=det.y, frame_start=det.frame, frame_end=det.frame, detections=[det]
            )
            active_wards[det.class_id].append(new_ward)
            next_ward_id += 1

        for class_id in list(active_wards.keys()):
            still_active = [w for w in active_wards[class_id] if det.frame - w.frame_end <= GAP_TOLERANCE]
            completed_wards.extend([w for w in active_wards[class_id] if det.frame - w.frame_end > GAP_TOLERANCE])
            active_wards[class_id] = still_active

    for class_id in active_wards:
        completed_wards.extend(active_wards[class_id])

    filtered = [w for w in completed_wards if w.detection_count >= MIN_FRAMES]
    for i, w in enumerate(sorted(filtered, key=lambda x: x.frame_start), 1):
        w.ward_id = i
    return sorted(filtered, key=lambda x: x.frame_start)

# === 保存関数 ===
def save_detections(detections: List[Detection], path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['frame', 'class_id', 'class_name', 'x', 'y', 'w', 'h', 'confidence'])
        for d in detections:
            writer.writerow([d.frame, d.class_id, d.class_name,
                           f"{d.x:.6f}", f"{d.y:.6f}", f"{d.w:.6f}", f"{d.h:.6f}", f"{d.confidence:.4f}"])

def save_wards(wards: List[Ward], path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['ward_id', 'class_id', 'class_name', 'x', 'y', 'frame_start', 'frame_end', 'detection_count', 'confidence_avg'])
        for w in wards:
            writer.writerow([w.ward_id, w.class_id, w.class_name, f"{w.x:.6f}", f"{w.y:.6f}",
                           w.frame_start, w.frame_end, w.detection_count, f"{w.confidence_avg:.4f}"])

print("関数定義完了")

In [ ]:
#cell-4: タイムライン統合関数

def load_frame_timestamps(path: Path) -> Optional[Dict[int, int]]:
    if not path.exists():
        return None
    frame_to_time = {}
    with open(path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            frame = int(row['frame_number'])
            time_ms = int(row['game_time_ms'])
            if time_ms >= 0:
                frame_to_time[frame] = time_ms
    return frame_to_time if frame_to_time else None

def timestamp_to_frame_from_map(ts: int, ft: Dict[int, int]) -> int:
    best_frame, best_diff = 0, float('inf')
    for frame, time_ms in ft.items():
        diff = abs(time_ms - ts)
        if diff < best_diff:
            best_diff, best_frame = diff, frame
    return best_frame

def load_ward_events(timeline_path: Path, ft: Optional[Dict[int, int]], ms_per_frame: float) -> Tuple[List[TimelineWardEvent], List[TimelineWardEvent], dict]:
    with open(timeline_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    placed, killed = [], []
    stats = {"total": 0, "filtered": {"creator_id_zero": 0, "undefined": 0}, "valid": 0}
    tid = 1

    for frame in data.get("info", {}).get("frames", []):
        for event in frame.get("events", []):
            if event.get("type") == "WARD_PLACED":
                stats["total"] += 1
                cid = event.get("creatorId", 0)
                ts = event.get("timestamp", 0)
                wt = event.get("wardType", "UNDEFINED")
                if cid == 0:
                    stats["filtered"]["creator_id_zero"] += 1
                    continue
                if wt == "UNDEFINED":
                    stats["filtered"]["undefined"] += 1
                    continue
                stats["valid"] += 1
                team = "blue" if 1 <= cid <= 5 else "red"
                fe = timestamp_to_frame_from_map(ts, ft) if ft else int(ts / ms_per_frame)
                placed.append(TimelineWardEvent(tid, "PLACED", ts, wt, cid, team, fe))
                tid += 1
            elif event.get("type") == "WARD_KILL":
                kid = event.get("killerId", 0)
                ts = event.get("timestamp", 0)
                wt = event.get("wardType", "UNDEFINED")
                fe = timestamp_to_frame_from_map(ts, ft) if ft else int(ts / ms_per_frame)
                killed.append(TimelineWardEvent(0, "KILLED", ts, wt, kid, "unknown", fe))
    return placed, killed, stats

def filter_river_sights(wards: List[Ward], ft: Optional[Dict[int, int]], ms_per_frame: float) -> Tuple[List[Ward], int]:
    filtered, removed = [], 0
    for w in wards:
        if not w.is_stealth_ward:
            filtered.append(w)
            continue
        is_river = any(np.sqrt((w.x_pixel - rx)**2 + (w.y_pixel - ry)**2) <= RIVER_SIGHT_RADIUS for rx, ry in RIVER_SIGHT_POSITIONS)
        if not is_river:
            filtered.append(w)
            continue
        if ft:
            duration = ft.get(w.frame_end, 0) - ft.get(w.frame_start, 0)
        else:
            duration = (w.frame_end - w.frame_start) * ms_per_frame
        if RIVER_SIGHT_DURATION_MIN_MS <= duration < RIVER_SIGHT_DURATION_MAX_MS:
            removed += 1
        else:
            filtered.append(w)
    return filtered, removed

def match_wards_hungarian(placed: List[TimelineWardEvent], killed: List[TimelineWardEvent],
                          wards: List[Ward], ft: Optional[Dict[int, int]], ms_per_frame: float) -> List[MatchedWard]:
    result = []
    rid = 1

    for wtype in ["stealth", "control"]:
        events = [e for e in placed if (wtype == "stealth" and e.is_stealth_ward) or (wtype == "control" and e.is_control_ward)]
        dets = [w for w in wards if not w.matched and ((wtype == "stealth" and w.is_stealth_ward) or (wtype == "control" and w.is_control_ward))]

        if not events or not dets:
            for e in events:
                cn = "control_ward" if e.is_control_ward else "stealth_ward"
                cn += "_enemy" if e.team == "red" else ""
                result.append(MatchedWard(rid, e.timeline_ward_id, cn, e.ward_type, e.team, -1, -1, -1.0, -1.0,
                              e.frame_expected, -1, 0.0, e.participant_id, e.timestamp, None, "timeline_only"))
                rid += 1
            continue

        INF = 1e9
        cost = np.full((len(events), len(dets)), INF)
        for i, e in enumerate(events):
            for j, w in enumerate(dets):
                fd = w.frame_start - e.frame_expected
                if fd < -FRAME_TOLERANCE or fd > FRAME_TOLERANCE * 3:
                    continue
                if w.team != e.team:
                    continue
                cost[i, j] = abs(fd) - w.confidence_avg * 10

        row_ind, col_ind = linear_sum_assignment(cost)
        matched_e, matched_d = set(), set()

        for i, j in zip(row_ind, col_ind):
            if cost[i, j] < INF:
                e, w = events[i], dets[j]
                w.matched = True
                matched_e.add(i)
                matched_d.add(j)
                result.append(MatchedWard(rid, e.timeline_ward_id, w.class_name, e.ward_type, e.team,
                              w.x_pixel, w.y_pixel, w.x, w.y, w.frame_start, w.frame_end, w.confidence_avg,
                              e.participant_id, e.timestamp, None, "matched"))
                rid += 1

        for i, e in enumerate(events):
            if i not in matched_e:
                cn = "control_ward" if e.is_control_ward else "stealth_ward"
                cn += "_enemy" if e.team == "red" else ""
                result.append(MatchedWard(rid, e.timeline_ward_id, cn, e.ward_type, e.team, -1, -1, -1.0, -1.0,
                              e.frame_expected, -1, 0.0, e.participant_id, e.timestamp, None, "timeline_only"))
                rid += 1

    for w in wards:
        if not w.matched and w.confidence_avg >= MIN_CONFIDENCE_DETECTION_ONLY:
            ts = ft.get(w.frame_start, 0) if ft else int(w.frame_start * ms_per_frame)
            wt = "CONTROL_WARD" if w.is_control_ward else "SIGHT_WARD"
            result.append(MatchedWard(rid, None, w.class_name, wt, w.team, w.x_pixel, w.y_pixel,
                          w.x, w.y, w.frame_start, w.frame_end, w.confidence_avg, None, ts, None, "detection_only"))
            rid += 1

    return result

def save_matched_wards(wards: List[MatchedWard], path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['ward_id', 'timeline_ward_id', 'class_name', 'ward_type', 'team',
                        'x_pixel', 'y_pixel', 'x_normalized', 'y_normalized',
                        'frame_start', 'frame_end', 'confidence_avg',
                        'creator_id', 'timestamp_placed', 'timestamp_killed', 'match_status'])
        for w in wards:
            writer.writerow([w.ward_id, w.timeline_ward_id or '', w.class_name, w.ward_type, w.team,
                           w.x_pixel if w.x_pixel >= 0 else '', w.y_pixel if w.y_pixel >= 0 else '',
                           f"{w.x_normalized:.6f}" if w.x_normalized >= 0 else '',
                           f"{w.y_normalized:.6f}" if w.y_normalized >= 0 else '',
                           w.frame_start, w.frame_end if w.frame_end >= 0 else '',
                           f"{w.confidence_avg:.4f}" if w.confidence_avg > 0 else '',
                           w.creator_id or '', w.timestamp_placed or '', w.timestamp_killed or '', w.match_status])

print("タイムライン統合関数定義完了")

In [ ]:
#cell-5: 全試合バッチ処理（YOLO推論 + クラスタリング + タイムライン統合）
match_dirs = sorted(DATASET_DIR.glob("JP1-*"))
total_matches = len(match_dirs)

print(f"全{total_matches}試合の処理を開始")
print(f"バッチサイズ: {BATCH_SIZE}")
print("="*60)

cumulative_matched = 0
cumulative_total = 0
results = []
start_time = time.time()

for i, match_dir in enumerate(tqdm(match_dirs, desc="全体進捗")):
    match_id = match_dir.name
    match_id_num = match_id.replace("JP1-", "")

    try:
        # 1. YOLO推論
        detections = run_inference(model, match_dir)
        if not detections:
            results.append({"match": match_id, "detections": 0, "wards": 0, "matched": 0})
            continue

        # 2. 保存
        save_detections(detections, match_dir / "detections_raw.csv")

        # 3. クラスタリング
        wards = cluster_detections(detections)
        save_wards(wards, match_dir / "wards.csv")

        # 4. タイムライン統合
        timeline_path = TIMELINE_DIR / f"JP1_{match_id_num}.json"
        ft_path = match_dir / "frame_timestamps.csv"
        ft = load_frame_timestamps(ft_path)
        ms_per_frame = DEFAULT_MS_PER_FRAME

        if timeline_path.exists():
            placed, killed, stats = load_ward_events(timeline_path, ft, ms_per_frame)
            wards_filtered, _ = filter_river_sights(wards, ft, ms_per_frame)
            matched_wards = match_wards_hungarian(placed, killed, wards_filtered, ft, ms_per_frame)
            save_matched_wards(matched_wards, match_dir / "wards_matched.csv")

            matched_count = sum(1 for w in matched_wards if w.match_status == "matched")
            total_count = stats["valid"]
        else:
            matched_count, total_count = 0, 0

        cumulative_matched += matched_count
        cumulative_total += total_count
        match_rate = cumulative_matched / cumulative_total * 100 if cumulative_total > 0 else 0

        results.append({"match": match_id, "detections": len(detections), "wards": len(wards), "matched": matched_count})

        if (i + 1) % 10 == 0 or i == total_matches - 1:
            tqdm.write(f"[{i+1}/{total_matches}] 累積マッチング率: {match_rate:.1f}%")

    except Exception as e:
        tqdm.write(f"エラー [{match_id}]: {e}")
        results.append({"match": match_id, "error": str(e)})

total_time = time.time() - start_time

print("\n" + "="*60)
print("処理完了")
print("="*60)
print(f"処理試合数: {len(results)}")
print(f"総検出数: {sum(r.get('detections', 0) for r in results)}")
print(f"総ward数: {sum(r.get('wards', 0) for r in results)}")
print(f"マッチング率: {cumulative_matched}/{cumulative_total} ({match_rate:.1f}%)")
print(f"総処理時間: {total_time/60:.1f}分")

In [ ]:
#cell-6: 結果確認
print("=== 処理結果サマリー ===")

match_results = []
for match_dir in match_dirs:
    matched_csv = match_dir / "wards_matched.csv"
    if matched_csv.exists():
        df = pd.read_csv(matched_csv)
        match_results.append({
            "match": match_dir.name,
            "wards": len(df),
            "matched": len(df[df["match_status"] == "matched"]) if "match_status" in df.columns else 0,
        })

results_df = pd.DataFrame(match_results)
print(f"\n処理済み試合: {len(results_df)}")
print(f"総ward数: {results_df['wards'].sum()}")
print(f"マッチ済み: {results_df['matched'].sum()}")

if len(results_df) > 0 and results_df['wards'].sum() > 0:
    final_match_rate = results_df['matched'].sum() / results_df['wards'].sum() * 100
    print(f"最終マッチング率: {final_match_rate:.1f}%")

results_df.head(10)

In [ ]:
#cell-7: グリッド特徴量生成（ward_grid.npz）

# === グリッド生成設定 ===
GRID_SIZE = 32
MINIMAP_SIZE = 512
CELL_SIZE = MINIMAP_SIZE // GRID_SIZE
NUM_PHASES = 3

TIME_PHASES = [
    (0, 10 * 60 * 1000),              # Phase 0: 0-10分
    (10 * 60 * 1000, 20 * 60 * 1000),  # Phase 1: 10-20分
    (20 * 60 * 1000, None),           # Phase 2: 20分以降
]

DEFAULT_DURATION_MS = {
    "YELLOW_TRINKET": 90 * 1000,
    "SIGHT_WARD": 90 * 1000,
    "CONTROL_WARD": 180 * 1000,
    "BLUE_TRINKET": 60 * 1000,
}

def calculate_duration_ms(ward_type: str, timestamp_placed: int, timestamp_killed) -> int:
    if timestamp_killed is not None and timestamp_killed != '':
        return int(timestamp_killed) - timestamp_placed
    return DEFAULT_DURATION_MS.get(ward_type, 90 * 1000)

def distribute_to_phases(timestamp_placed: int, duration_ms: int):
    results = []
    ward_start = timestamp_placed
    ward_end = timestamp_placed + duration_ms
    
    for phase_idx, (phase_start, phase_end) in enumerate(TIME_PHASES):
        if phase_end is None:
            phase_end = float('inf')
        overlap_start = max(ward_start, phase_start)
        overlap_end = min(ward_end, phase_end)
        if overlap_start < overlap_end:
            duration_sec = (overlap_end - overlap_start) / 1000.0
            results.append((phase_idx, duration_sec))
    return results

def generate_ward_grid(wards_csv_path: Path) -> dict:
    blue_grid = np.zeros((NUM_PHASES, GRID_SIZE, GRID_SIZE), dtype=np.float32)
    red_grid = np.zeros((NUM_PHASES, GRID_SIZE, GRID_SIZE), dtype=np.float32)
    match_id = wards_csv_path.parent.name
    
    with open(wards_csv_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row.get('match_status') == 'timeline_only':
                continue
            
            x_str, y_str = row.get('x_pixel', ''), row.get('y_pixel', '')
            if not x_str or not y_str:
                continue
            
            x_pixel, y_pixel = int(float(x_str)), int(float(y_str))
            if not (0 <= x_pixel < MINIMAP_SIZE and 0 <= y_pixel < MINIMAP_SIZE):
                continue
            
            team = row.get('team', '')
            if team not in ('blue', 'red'):
                continue
            
            ts_str = row.get('timestamp_placed', '')
            if not ts_str:
                continue
            timestamp_placed = int(float(ts_str))
            
            timestamp_killed = row.get('timestamp_killed', '')
            ward_type = row.get('ward_type', 'SIGHT_WARD')
            
            duration_ms = calculate_duration_ms(ward_type, timestamp_placed, timestamp_killed if timestamp_killed else None)
            grid_x = min(x_pixel // CELL_SIZE, GRID_SIZE - 1)
            grid_y = min(y_pixel // CELL_SIZE, GRID_SIZE - 1)
            
            for phase_idx, duration_sec in distribute_to_phases(timestamp_placed, duration_ms):
                if team == 'blue':
                    blue_grid[phase_idx, grid_y, grid_x] += duration_sec
                else:
                    red_grid[phase_idx, grid_y, grid_x] += duration_sec
    
    return {"blue": blue_grid, "red": red_grid, "match_id": match_id}

# グリッド生成実行
print("グリッド特徴量生成を開始...")
print("="*60)

grid_success = 0
grid_skip = 0

for match_dir in tqdm(match_dirs, desc="グリッド生成"):
    wards_csv = match_dir / "wards_matched.csv"
    output_path = match_dir / "ward_grid.npz"
    
    if not wards_csv.exists():
        grid_skip += 1
        continue
    
    try:
        grid_data = generate_ward_grid(wards_csv)
        np.savez(output_path, blue=grid_data["blue"], red=grid_data["red"], match_id=grid_data["match_id"])
        grid_success += 1
    except Exception as e:
        tqdm.write(f"エラー [{match_dir.name}]: {e}")
        grid_skip += 1

print("\n" + "="*60)
print(f"グリッド生成完了: {grid_success}試合")
print(f"スキップ: {grid_skip}試合")

In [ ]:
#cell-8: 処理完了
print("="*60)
print("Phase 3 完了（Colab版）")
print("="*60)
print(f"\n出力ファイル（Google Driveに保存）:")
print(f"  - detections_raw.csv: 生の検出結果")
print(f"  - wards.csv: クラスタリング後ward情報")
print(f"  - wards_matched.csv: タイムライン統合済み")
print(f"  - ward_grid.npz: グリッド特徴量（32x32x3時間帯）")
print(f"\n保存先: {DATASET_DIR}/JP1-*/")
print("\n次のステップ:")
print("  -> 07_vision_score.ipynb でモデル学習・ヒートマップ可視化")